Testing Individual Attri

In [1]:
"Set working directory"
import os
os.chdir('../')

In [2]:
# agent_cm
import json
import numpy as np
import os
import pandas as pd


from math import exp, log
from typing import List, Tuple
from tqdm import tqdm

from agentpy_cm.objects import Object
from agentpy_cm.agent import Agent
#from agentpy_cm.sequences import AgentList_____________Remove
#from agentpy_cm.tools import AgentpyError, make_list





In [3]:
from agentpy_cm.hypothesis import PARAMS_INDIVIDUAL, \
    PARAMS_IPF_WEIGHTS, Hypothesis
from datetime import datetime
from agentpy_cm.datadict import DataDict
from agentpy_cm.sample import Range, Values
from agentpy_cm.grid import Grid
from agentpy_cm.space import Space
from agentpy_cm.tools import AttrDict, AgentpyError, make_list, InfoStr
from agentpy_cm.sequences import AgentList
from agentpy_cm.model import Model
# from .agent_cm import Individual, Populating ____________for SocPD file
#from agentpy_cm.model_cm import SocPD # ____________only for runing final notwbook


#comma

from typing import List, Tuple
from tqdm import tqdm

In [38]:
class Individual(Agent):

    def setup(self):
        # linking with model #--> Agent
        
        self.random = self.model.random
        self.status_quo = self.model.status_quo
        #self.Hypothesis_settings = self.p.Hypothesis_settings #______________removed
        
        #Grid method
        self.grid = self.model.grid
        
        #local variable
        #self.Hypothesis_settings = None #______________removed
        self.status= False
        self.moving= False
        self.features = None
        self.influencing_profile : pd.DataFrame
        self.influenced_score :  pd.Series
        self.move_to_pos : Tuple(int,int)
        self.share_similar_diet :float = .0
    
        #Hypo_dict = self.p.Hypothesis_settings #_____________ removed
        #Hypothesis.read_hypotheses(Hypo_dict)#__________________ removed

        #call-out hypothesis on actions #__________________changed to model.....

        self.env_beta = self.model.env_beta
        self.actions = self.model.all_possible_actions
        self.status_var = self.model.status_var
        self.pos_nw_actions = self.model.pos_nw_actions 
        self.neg_nw_actions = self.model.neg_nw_actions       
        self.decisions_to_move = self.model.decisions_to_move 
        self.decisions_to_adopt = self.model.decisions_to_adopt
        self.actions_to_nw = self.model.actions_to_nw  
        self.actions_to_self = self.model.actions_to_self
        self.rules = self.model.data_dfs

        
        # parameters for populating
        #self._pop : int = None ---> attach with model
        #self._dir_params = self.p.dir_params

        # get nw-influencin actions that decides moving from hypothesis
        self.nw_decisions_to_move = list(set(self.decisions_to_move).intersection(set(self.actions_to_nw)))
        self.self_decisions_to_move = list(set(self.decisions_to_move).intersection(set(self.actions_to_self)))
    
    @staticmethod
    def get_p(sum_betas:float) -> float:
        '''logit reversed'''
        return exp(sum_betas)/(1+exp(sum_betas))
    
    '''step 1 :set up agents's status'''
    #TESTED 
    def get_status_step0(self) -> None: 

        fs = self.features
        self.status = fs[f'{self.status_var}_yes'] ==1 

    #TESTED
    def scoring_influenced_self(self) ->pd.Series: 
        '''collect influenced score from self '''
        params_self = self.rules['actions_to_self']
        #actions = params_self.index___________REMOVE_________
        _score_self = params_self.dot(self.features) 
        return _score_self #_______________EDIT_______
    #TESTED
    def set_influencing_profile(self) -> pd.DataFrame:
        
        params_nw = self.rules['actions_to_nw']
        features = self.features  
        #ncols = features.shape[0] _____________REMOVE_______

        # Patching to get diagonal mtx of agent's features 
        #patched_profl = np.zeros((ncols, ncols))___________REMOVE_________
        #patched_profl = np.fill_diagonal(patched_profl, np.array(features).flatten())__________REMOVE_____
        # match action params to get agent's influenxing profile
        influencing_profile = params_nw.mul(features)
         
        # zero-out the influence doesn't match agent's status
        if self.status:
            influencing_profile.loc[self.neg_nw_actions] = 0
        else:
            influencing_profile.loc[self.pos_nw_actions] = 0

        #_________changed
        self.influencing_profile = influencing_profile
        
    
    # Update adoption status based on score
    def update_status(self) -> bool: #_______________changed
        _score = sum(self.influenced_score[self.decisions_to_adopt])
        _score += self.env_beta + self.status_quo
        return self.random.random() <= self.get_p(_score)
    #_____________NOTE_______: changing enviromental beta
    def update_agent_combined(self):
        #-----------------------
        # Testing individual computation in comma-main/../demo_individual
        _score_self = self.scoring_influenced_self()
        
        ########## 2. Get scores of network-influenced ---- HOMOPHILY
            ## UPDATE self influencing-profile, preparing to influence neighbor
        # _______________removed_________________self.influencing_profile = self.set_influencing_profile()   
        # if influencing profile need to be update the first time at global level to make sure neighbors have influencing'sprofile updated at step 0 
            
            ## Get network influenced scores from neighbor's influencing-profiles
        neighbors = self.grid.neighbors(self)
        if len(neighbors)>0:
            neigh_prfs = [n.influencing_profile for n in neighbors]
            
                ## WARNING :Method HOMOPHILY - 
            result_nw = [p.dot(self.features) for p in neigh_prfs] #_________________changed 

            ### Method get lowest score in _score_to_move to defind weakest-tie neighbors
                ### filter to get decision to move from network's influences, then sum score by neighbor
                ### then get sum score from both self and nw decisions
            #mask = np.isin(self.actions_to_nw, self.nw_decisions_to_move)_____________removed

            #@TESTED________
            #______________________changed 
            _score_to_move =[sum(s[self.nw_decisions_to_move]) \
                            + sum(_score_self[self.self_decisions_to_move]) \
                                for s in result_nw] 
                ### Access neighbor with weakest ties
            
            ### Decision to move away from lowest score neighbor - simple method: run random
            
            # TESTED_____
            neigh_pos : Tuple(int,int) =  None
            for i, nb in enumerate(neighbors):
                if i == np.argmin(_score_to_move):
                    neigh_pos = self.grid.positions[nb]
                    choice = [2,-2]
                    path = (self.random.choice(choice), self.random.choice(choice))
                    self.move_to_pos = tuple([p + c for p, c in zip(path, neigh_pos)])
                break
            #____________changed
            prob_move = self.get_p(-np.min(_score_to_move)) # reversed affect 
        else:
            result_nw = [pd.Series(0, index = self.actions_to_nw)]
            prob_move = self.get_p(-sum(_score_self[self.self_decisions_to_move]))
            self.move_to_pos = self.random.choice(self.model.grid.empty)
            

        self.moving = self.random.random() <= prob_move

        ### summary all action score of each agent
        _score_nw = pd.Series(sum(result_nw))
        
        self.influenced_score = pd.concat([_score_self,_score_nw])
        self.status = self.update_status()
        
        # For get_segregation
        similar = len([n for n in neighbors if n.status == self.status])
        ln = len(neighbors)
        self.share_similar_diet = similar / ln if ln > 0 else 0
   
    '''STEP 4 : taking action following status'''
    def find_new_friends(self):
        self.grid.move_to(self, self.move_to_pos)
        
    def update_agent_profile_by_status(self): #_______________changed
        if self.status:
            self.features[self.features.index.str.contains(pat = f'{self.status_var}')] = [0,1]
        else:
            self.features[self.features.index.str.contains(pat = f'{self.status_var}')] = [1,0]    
     

In [39]:
class Populating(Object):
    def __init__(self, model):
        super().__init__(model)
        self.nprandom = model.nprandom 
        self.all_possible_features = self.model.all_possible_features#__________________change this
        #self.dir_params = Hypothesis.dir_params #_________________removed
    
    def sampling_from_ipf(self, dir_params, pop:int) -> pd.DataFrame: #_____________changed
        """
        Sample from IPF distribution saved
        as `weights.csv` in the parameters folder

        Parameters
        ----------
        pop (int): size of data sample
        dir_params (str): path to the parameters folder

        Returns
        -------
        sample (pandas.dataFrame): dataframe containing the sampling
        """
        fpath_weights = os.path.join(dir_params, PARAMS_IPF_WEIGHTS)#____________changed
        assert os.path.isfile(fpath_weights)
        df_weights = pd.read_csv(fpath_weights, sep=",", index_col=0)
        weights = df_weights["weight"] / df_weights["weight"].sum()
        indices = df_weights.index
        
        #______apply seed on numpy -> change np.random_____________________
        sample_indices = self.nprandom.choice(indices, pop, p=weights) 
        sample = df_weights.loc[sample_indices].drop(["weight"], axis=1)
        sample = sample.reset_index(drop=True)
        return sample

    def populate_ipf(self, dir_params, pop:int) -> List:  #_______________changed 
        """
        Create a population of individual agents
        with the given weights obtained via IPF

        Args:
            pop (int): size of data sample.
            dir_params (str): path to parameters folder.

        Returns:
            List[Individual]: A list containing instances of
            the individual class, each representing an
            agent with specific features.
        """
        _features = pd.DataFrame()

        sample = self.sampling_from_ipf(dir_params, pop)#_______________added

        # one-hot encoding
        encoded_columns = pd.get_dummies(sample).reindex(
            columns=self.all_possible_features,
            fill_value=0
        )
        _features = pd.concat([_features, encoded_columns], axis=1)
        #________________added
        cols = sorted(_features.columns)
        _features = _features[cols]

        # Add 'baseline' column filled with ones if this is not present yet
        if 'baseline' not in _features.columns:
            _features.insert(0, "baseline", 1)

        return [_features.iloc[i] for i in
                tqdm(range(pop), desc="Populating individuals", unit="i")]

    def populate_simple(self, dir_params, pop:int) -> List:
        """
        Create a population of individual agents
        with the given feature parameters.

        Args:
            pop (int): population size, i.e., number of agents.
            dir_params (str): dir to the folder containing
            feature parameter file.
            #from_scratch (bool, optional): flag of creating hypothesis
            from scratch or reading from files. Defaults to False.

        Returns:
            list[Individual]: a list of Individual agents
        """
        assert pop > 0, 'Size must be positive!'
        assert isinstance(pop, int), 'Pop - population size must be integer!'
        assert os.path.isdir(dir_params), "Given folder doesn't exist!"
        #_________________changed
        fpath_params_individual = os.path.join(dir_params, PARAMS_INDIVIDUAL) #_______________changed
        with open(fpath_params_individual) as f:
            features = json.load(f)

        _features = pd.DataFrame()
        for feature, distribution in features.items():
            _features[feature] = self.nprandom.choice(
                distribution[0], pop, p=distribution[1]
            )

            # Define all possible columns (including those not in the sample)
            # When the sample size is too small,
            # this doesn't cover all categories,
            # the resulting DataFrame thus lacks those columns.
            # To solve the issue, we ensure all possible categories are present
            # when creating the dummy variables

        # one-hot encoding
        categorical_cols = _features.select_dtypes(include=['object'])
        encoded_cols = pd.get_dummies(categorical_cols).reindex(
            columns=self.all_possible_features,
            fill_value=0
        )
        _features.drop(categorical_cols.columns, axis=1, inplace=True)
        _features = pd.concat([_features, encoded_cols], axis=1)
         #________________added
        cols = sorted(_features.columns)
        _features = _features[cols]

        # Add 'baseline' column filled with ones
        _features.insert(0, "baseline", 1)

        return [_features.iloc[i] for i in
                tqdm(range(pop), desc="Populating individuals", unit="i")]
    



In [50]:

class SocPD(Model, Hypothesis):  
    def setup(self) :
                # Private variables

        # Set-up Hypothesis 
        Hypo_dict = self.p.Hypothesis_settings #_____________ added
        Hypothesis.validate_n_read_hypotheses(Hypo_dict)#__________________added

        #call-out hypothesis on actions
        self._dir_params = Hypothesis.dir_params
        self.env_beta = Hypothesis.env_beta
        self.actions = Hypothesis.all_possible_actions
        self.status_var = Hypothesis.status_var
        self.pos_nw_actions = Hypothesis.pos_nw_actions 
        self.neg_nw_actions = Hypothesis.neg_nw_actions       
        self.decisions_to_move = Hypothesis.decisions_to_move 
        self.decisions_to_adopt = Hypothesis.decisions_to_adopt
        self.actions_to_nw = Hypothesis.actions_to_nw  
        self.actions_to_self = Hypothesis.actions_to_self
        self.rules = Hypothesis.data_dfs

        # Set up simple model's params
        self.status_quo = 0
        self._use_ipf : None = bool
        pop = self.pop = self.p['pop']
        den = self.p['den']
        s = int(np.ceil(np.sqrt(pop/den)))

        if 'use_ipf' in self.p:
            self._use_ipf = self.p['use_ipf']
        else:
            self._use_ipf = False  #_____________ changed
        self.report('use_ipf', self._use_ipf) #___________changed
        #self.Hypothesis_settings = self.p.Hypothesis_settings
        
        #____________GRID_______________________________
        # setup model's Grid
        self.grid = Grid(self, (s, s), track_empty=True)
        #____________AGENTS________________________________
        # generate agents
        self.agents = AgentList(self, pop, Individual) 
        # generate features then update agent's features and status
        self.Populating = Populating(self)
        if self._use_ipf: #___________ changed
            _feature_iter = self.Populating.populate_ipf(self._dir_params, pop)
        else:
            _feature_iter = self.Populating.populate_simple(self._dir_params, pop)        
            # update agent's features 
        for i, a in enumerate(self.agents):
            a.features = _feature_iter[i]
            # update Agent status based on feature generation        
        self.agents.get_status_step0()
        #____________ADD AGENTS ON THE GRID_______________
        self.grid.add_agents(self.agents, random=True, empty=True)


    def update(self): 
        ''' TESTING updated status and profile
        for i, a in enumerate(self.agents):
            if i ==4:
                print(f'step: {self.t}')
                print(f'current status : {a.status}')
        '''
        positive_agents = self.agents.select(self.agents.status==True)
        self.status_quo = len(positive_agents)/self.pop

        # Stop simulation if all are positive or negative
        if self.status_quo == 0 or self.status_quo == 1:
            self.stop()
        #record status
        self['positive'] = self.status_quo
        self.record('positive')
        self['negative'] = 1 - self['positive']
        self.record('negative')
        
        # update from t= 0
        self.agents.set_influencing_profile()  #___________ADDED
        self.agents.update_agent_combined()
        self.unhappy = self.agents.select(self.agents.moving == True)
        ''' TESTING updated status and profile
        for i, a in enumerate(self.agents):
            if i ==4:
                print('current profile')
                print(a.features[a.features.index.str.contains(pat = f'{self.status_var}')])
                print(f'updated status : {a.status}')

                print()
        '''
            

    def step(self):
        self.unhappy.find_new_friends()
        self.agents.update_agent_profile_by_status()
    
    def get_segregation(self):
        # Calculate average percentage of similar neighbors
        return round(sum(self.agents.share_similar_diet) / self.pop, 2)

    def end(self):
        # Measure segregation at the end of the simulation
        self.report('segregation', self.get_segregation())


In [51]:
dir_params = 'parameters_demo'
all_possible_actions=  ['veg_prob', 
                        'influencing_Veg', 
                        'influencing_Om', 
                        'nw_satisfaction', 
                        'making_effort', 
                        'environment_veg']

Hypothesis_settings = { 'dir_params' : dir_params ,
                        'status_var':'Depressed',
                        'all_possible_actions':all_possible_actions,
                        'actions_to_self':['veg_prob', 'nw_satisfaction','environment_veg'],
                        'actions_to_nw':['influencing_Veg', 'influencing_Om','making_effort'],
                        'pos_nw_actions':['influencing_Veg'],
                        'neg_nw_actions':['influencing_Om'],
                        'decisions_to_move':['nw_satisfaction', 'making_effort',  ],
                        'decisions_to_adopt':['veg_prob', 'influencing_Veg', 'influencing_Om', 'environment_veg'], 
                        'env_beta': -0.8}


parameters = {'Hypothesis_settings' : Hypothesis_settings,
            'pop' : 15,
            'den': .75,
            'steps': 10,
            'seed' : 234,
            'use_ipf': True,
            } 

In [52]:

model = SocPD(parameters)
model.run()

Populating individuals: 100%|██████████| 15/15 [00:00<00:00, 14898.07i/s]

step: 0
current status : False
current profile
depressed_no     1
depressed_yes    0
Name: 4, dtype: int64
updated status : True

step: 1
current status : True
current profile
depressed_no     0
depressed_yes    1
Name: 4, dtype: int64
updated status : True

Completed: 1 stepsstep: 2
current status : True
current profile
depressed_no     0
depressed_yes    1
Name: 4, dtype: int64
updated status : True

Completed: 2 stepsstep: 3
current status : True
current profile
depressed_no     0
depressed_yes    1
Name: 4, dtype: int64
updated status : False

Completed: 3 stepsstep: 4
current status : False
current profile
depressed_no     1
depressed_yes    0
Name: 4, dtype: int64
updated status : True

Completed: 4 stepsstep: 5
current status : True
current profile
depressed_no     0
depressed_yes    1
Name: 4, dtype: int64
updated status : True

Completed: 5 stepsstep: 6
current status : True
current profile
depressed_no     0
depressed_yes    1
Name: 4, dtype: int64
updated status : True

Comp

DataDict {
'info': Dictionary with 9 keys
'parameters': 
    'constants': Dictionary with 6 keys
'variables': 
    'SocPD': DataFrame with 2 variables and 11 rows
'reporters': DataFrame with 3 variables and 1 row
}

In [16]:
model.reporters

{'seed': 42, 'use_ipf': True, 'segregation': 0.51}

In [ ]:
def animation_plot(model, ax):
    group_grid = model.grid.attr_grid('group')
    ap.gridplot(group_grid, cmap='Accent', ax=ax)
    ax.set_title(f"Segregation model \n Time-step: {model.t}, "
                 f"Segregation: {model.get_segregation()}")

fig, ax = plt.subplots()
model = SegregationModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())